In [6]:
import pandas as pd
import torch
from preprocessing import *
import time
from PC_transformer import PCTransformer

set_seed(42)

In [7]:
df = pd.read_csv(r'output_csv\HCMUT-SuperNodeXP-2017-1.0.swf.csv')
# df = df.head(100)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15886 entries, 0 to 15885
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    15886 non-null  float64
 1   submit_time               15886 non-null  float64
 2   wait_time                 15886 non-null  float64
 3   run_time                  15886 non-null  float64
 4   num_allocated_processors  15886 non-null  float64
 5   avg_cpu_time_used         15886 non-null  float64
 6   used_memory               15886 non-null  float64
 7   requested_processors      15886 non-null  float64
 8   requested_time            15886 non-null  float64
 9   requested_memory          15886 non-null  float64
 10  status                    15886 non-null  float64
 11  user_id                   15886 non-null  float64
 12  group_id                  15886 non-null  float64
 13  executable_id             15886 non-null  float64
 14  queue_

In [8]:
df.head()

,job_id,submit_time,wait_time,run_time,num_allocated_processors,avg_cpu_time_used,used_memory,requested_processors,requested_time,requested_memory,status,user_id,group_id,executable_id,queue_id,partition_id,preceding_job_id,think_time
0,4.0,0.0,0.0,13.0,24.0,7.0,14761.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
1,8.0,40819.0,0.0,61.0,24.0,45.0,14761.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
2,10.0,45255.0,0.0,60.0,24.0,45.0,14760.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
3,11.0,45782.0,0.0,1.0,24.0,0.0,0.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
4,12.0,56510.0,0.0,1.0,24.0,0.0,0.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0


In [9]:
feature_columns = ['requested_processors', 'requested_time', 'avg_cpu_time_used', 'used_memory', 'submit_time', 'wait_time', 'user_id', 'group_id', 'executable_id', 'queue_id']
target_column = 'run_time'

# Hyperparameters
input_dim = len(feature_columns)    # Example input dimension
d_model = 64      # Model dimension
num_heads = 8     # Number of attention heads
d_ff = 256        # Feed-forward dimension
num_layers = 3    # Number of transformer blocks
output_dim = 1    # Output dimension
batch_size = 128  # Batch size
seq_len = 20      # Length of data group
num_epochs = 20   # Number of training epochs
dropout = 0.2    # Dropout rate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataloaders, scaler = prepare_data_seq(df, feature_columns, target_column, seq_len=seq_len, batch_size=batch_size)

In [10]:
start = time.time()

# Create model
model = PCTransformer(input_dim, d_model, num_heads, d_ff, num_layers, output_dim, dropout).to(device)

# Train the model
model.train_model(model, dataloaders, num_epochs, device, scaler)

end = time.time()

# Final evaluation on test set
test_rmse, test_mae, test_mse, test_r2 = model.evaluate_model(model, dataloaders['test'], device, scaler, input_dim)
print(f'\nFinal Test Metrics:')
print(f'Test RMSE: {test_rmse:.4f}, Test MAE: {test_mae:.4f}, Test MSE: {test_mse:.4f}, Test R2: {test_r2:.4f}')
print(f'Training time: {end - start:.2f} seconds')

Epoch [1/20]
Train Loss: 0.3755, Val Loss: 0.3583
Val RMSE: 151630.6982, Val MAE: 101977.8999, Val MSE: 22991868641.4527, Val R2: -0.5681
Epoch [2/20]
Train Loss: 0.2790, Val Loss: 0.2959
Val RMSE: 138842.3047, Val MAE: 90194.0286, Val MSE: 19277185562.1861, Val R2: -0.3147
Epoch [3/20]
Train Loss: 0.2740, Val Loss: 0.3261
Val RMSE: 145236.2224, Val MAE: 95857.2568, Val MSE: 21093560285.7519, Val R2: -0.4386
Epoch [4/20]
Train Loss: 0.2715, Val Loss: 0.3118
Val RMSE: 142288.1091, Val MAE: 93108.5612, Val MSE: 20245905999.2490, Val R2: -0.3808
Epoch [5/20]
Train Loss: 0.2689, Val Loss: 0.3043
Val RMSE: 140739.2156, Val MAE: 91633.0518, Val MSE: 19807526816.5680, Val R2: -0.3509
Epoch [6/20]
Train Loss: 0.2688, Val Loss: 0.2903
Val RMSE: 137784.4489, Val MAE: 89148.8784, Val MSE: 18984554363.1635, Val R2: -0.2948
Epoch [7/20]
Train Loss: 0.2679, Val Loss: 0.2637
Val RMSE: 131703.4188, Val MAE: 85338.0081, Val MSE: 17345790523.2209, Val R2: -0.1830
Epoch [8/20]
Train Loss: 0.2673, Val Los